In [1]:
pip install mlxtend

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 14.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 30.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import trino

# Configuring Data Mesh
USER = 'fernando.jimenez@glovoapp.com'
HOST = 'starburst.g8s-data-platform-prod.glovoint.com'
PORT = 443

conn_details = {
    'host': HOST,
    'port': PORT,
    'user': USER,
    'http_scheme': 'https',
    'auth': trino.auth.OAuth2Authentication()
}


# Defining the function we'll use to extract the data
def sql_dl(query: str) -> pd.DataFrame:
    # print('Start time: ' + str(datetime.datetime.now()))
    conn = trino.dbapi.connect(**conn_details)
    cur = conn.cursor()
    cur.execute(query)
    rows = cur.fetchall()
    cols = [i[0] for i in cur.description]
    df = pd.DataFrame(rows, columns=cols)
    # print('End time: ' + str(datetime.datetime.now()))
    return df

In [2]:

query = """
SELECT 
    -- p_creation_date,
    -- order_id,
    -- customer_id,
    --list_of_name,
    list_of_product_brand,
    -- count(order_id) as count_orders
    order_id
    -- *
FROM 
(
    SELECT
        bought_products.order_id AS order_id,
        order_descriptors.customer_id AS customer_id,
        DATE_FORMAT(bought_products.p_creation_date, '%Y-%m-%d') AS p_creation_date,
        ARRAY_JOIN(ARRAY_SORT(ARRAY_DISTINCT(ARRAY_AGG(bought_products.product_name))), ',') AS "list_of_name",
        ARRAY_JOIN(ARRAY_SORT(ARRAY_DISTINCT(ARRAY_AGG(catalog_product_details.product_brand))), ',') AS "list_of_product_brand",
        ARRAY_JOIN(ARRAY_SORT(ARRAY_DISTINCT(ARRAY_AGG(catalog_product_details.product_category_level_two))), ',') AS "list_of_level_2"
    FROM delta.central__bought_products_looker__odp.bought_products AS bought_products
    LEFT JOIN delta.central_order_descriptors_odp.order_descriptors_v2 AS order_descriptors 
        ON order_descriptors.order_id = bought_products.order_id
        AND order_descriptors.p_creation_date = bought_products.p_creation_date
    LEFT JOIN delta.mfc_inventory_odp.products_v2 AS catalog_product_details 
        ON CONCAT(CAST(bought_products.store_address_id AS VARCHAR), '_', bought_products.product_external_id) = 
           CONCAT(CAST(catalog_product_details.store_address_id AS VARCHAR), '_', catalog_product_details.product_sku)
    WHERE
        order_descriptors.order_subvertical = 'MFC' 
        AND order_descriptors.order_country_code = 'ES'
        AND bought_products.p_creation_date >= TIMESTAMP '2024-10-01'
        AND bought_products.p_creation_date < DATE_ADD('month', 1, TIMESTAMP '2024-10-01')
    GROUP BY
        1, 2, 3
    ORDER BY
        1 DESC
)
WHERE 
    list_of_level_2 NOT LIKE '%Fruit%' 
    AND list_of_level_2 NOT LIKE '%Vegetables%' 
    AND list_of_product_brand NOT LIKE '%UNKNOWNBRAND%'
GROUP BY 
    1, 2
"""

# Llama a la función `sql_dl` con la consulta SQL
df = sql_dl(query)

# Muestra los datos obtenidos
print(df)


Open the following URL in browser for the external authentication:
https://starburst.g8s-data-platform-prod.glovoint.com/oauth2/token/initiate/dde6f4e126f87c4264624dbd8e57cedb8c505a6269d3c45a17c1a8b974602225
                                  list_of_product_brand      order_id
0                                            Antioqueño  100912640064
1                                                  Yay!  100920002855
2      Coca-Cola,Johnnie Walker,Mahou,Procubitos,Sprite  100910332713
3                      Aquarel,Florette,Navidul,Profeel  100948853965
4                                     Estrella Damm,J&B  100927159212
...                                                 ...           ...
85933                                          Viña Sol  100917542536
85934                                         Coca-Cola  100925442172
85935                                 Franui,Nupik,Pato  100920046754
85936                                      AUSONIA,Evax  100966436841
85937              Cas

In [3]:
df

,list_of_product_brand,order_id
0,Antioqueño,100912640064
1,Yay!,100920002855
2,"Coca-Cola,Johnnie Walker,Mahou,Procubitos,Sprite",100910332713
3,"Aquarel,Florette,Navidul,Profeel",100948853965
4,"Estrella Damm,J&B",100927159212
...,...,...
85933,Viña Sol,100917542536
85934,Coca-Cola,100925442172
85935,"Franui,Nupik,Pato",100920046754
85936,"AUSONIA,Evax",100966436841


In [4]:
 df.isna().sum()

list_of_product_brand    0
order_id                 0
dtype: int64

In [5]:
df = df.dropna()

In [6]:

from mlxtend.frequent_patterns import apriori, association_rules


# df = pd.read_csv('pedidos_completos.csv') 

# Separar las cadenas de texto en una lista de marcas
df['list_of_product_brand'] = df['list_of_product_brand'].apply(lambda x: x.split(','))

# Crear una representación binaria (cada columna es una marca, cada fila un pedido)
from mlxtend.preprocessing import TransactionEncoder

# Usamos TransactionEncoder para convertir la lista de marcas en formato adecuado
te = TransactionEncoder()
te_ary = te.fit(df['list_of_product_brand']).transform(df['list_of_product_brand'])
df_binary = pd.DataFrame(te_ary, columns=te.columns_)

# df_binary ahora tiene 1 si el producto fue comprado y 0 si no


In [7]:
df_binary

,,S.L.,18/70,1906,7Up,8.6,A Torres,ANETO,AT Roca,AUSONIA,...,Zarro,Zoco,Zzzquil,dakota,iHam,nestle_extrafino,oleguer,passport scotch,perrier jouet,Árabe
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85933,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
85934,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
85935,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
85936,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False


In [8]:
pd.set_option('display.max_rows', None)

In [ ]:
###FP GROWTH
from mlxtend.frequent_patterns import fpgrowth

# Aplicar FP-Growth con un soporte mínimo
frequent_itemsets = fpgrowth(df_binary, min_support=0.0001, use_colnames=True)

# Mostrar los conjuntos frecuentes
frequent_itemsets.head(10)
frequent_itemsets.sort_values(by='support', ascending=False).head(20)

In [10]:
from mlxtend.frequent_patterns import association_rules

# Generar reglas de asociación basadas en los conjuntos frecuentes
rules = association_rules(frequent_itemsets, metric="lift")

# Ordenar las reglas por lift
rules = rules.sort_values(by='lift', ascending=False)

# Mostrar las reglas más relevantes
rules.head(50)

TypeError: association_rules() missing 1 required positional argument: 'num_itemsets'

In [ ]:
from mlxtend.frequent_patterns import association_rules

# Generar reglas de asociación basadas en los conjuntos frecuentes
rules = association_rules(frequent_itemsets, metric="lift", num_itemsets=2)  # Ajusta num_itemsets según lo necesites

# Ordenar las reglas por lift
rules = rules.sort_values(by='lift', ascending=False)


Support

Support mide la frecuencia con la que ambos, los antecedents y los consequents, aparecen juntos en el conjunto de transacciones. Es decir, la proporción de transacciones donde ambos productos (antecedentes y consecuentes) fueron comprados juntos.

Por ejemplo, si "Coca-Cola" y "Patatas Torres" se compran juntas en el 5% de los pedidos, el support sería 0.05.

Confidence

Confidence mide la probabilidad de que los consequents se compren dado que los antecedents ya han sido comprados. Es decir, cuán a menudo se compran los consequents cuando los antecedentes están presentes.

Si la confidence es 0.70, significa que el 70% de las veces que alguien compra los antecedentes (p.ej. "Coca-Cola"), también compran los consequents (p.ej. "Patatas Torres").

Leverage

Leverage mide la diferencia entre el soporte real de los productos que se compran juntos y el soporte esperado si fueran independientes. Indica cuánto más (o menos) se compran juntos los productos comparado con lo que esperarías por azar.

Si el leverage es 0.02, significa que los productos se compran juntos un 2% más frecuentemente que lo que esperaríamos por azar.

Conviction

Conviction mide la dependencia de los consequents dado que se han comprado los antecedents, pero tiene en cuenta las transacciones donde la regla no es válida (es decir, cuando se compra el antecedente pero no el consecuente). Un valor mayor de convicción indica una fuerte relación entre los antecedentes y consecuentes.

Si el conviction es 1.5, significa que los antecedentes llevan a la compra de los consecuentes 1.5 veces más frecuentemente que lo esperado si fueran independientes.

Zhang's Metric

Zhang's Metric es una métrica que mide la importancia de una regla teniendo en cuenta la dependencia tanto de los antecedentes como de los consecuentes. Se considera una métrica robusta porque se ajusta a los problemas que presentan otras métricas como el lift. Zhang’s metric varía entre -1 y 1, donde los valores positivos indican una relación positiva, 0 indica independencia, y los valores negativos indican una relación negativa.

Este valor es útil cuando se quiere obtener una medida más equilibrada de la relación entre productos, ya que no está influenciado por la frecuencia como otras métricas.

In [14]:
import openpyxl
rules.to_excel('reglas_asociacion.xlsx', index = False)

In [12]:
filtered_rules_marca = rules[rules['consequents'].apply(lambda x: 'Mahou' in x)]

filtered_rules_marca = filtered_rules_marca.sort_values(by='confidence',ascending=False)

filtered_rules_marca.head(50)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
260,"(Coca-Cola, Procubitos)",(Mahou),0.016326,0.056797,0.001338,0.081967,1.443167,1.0,0.000411,1.027418,0.312176,0.018642,0.026686,0.052764
249,(Procubitos),(Mahou),0.050746,0.056797,0.003561,0.070167,1.235412,1.0,0.000679,1.014380,0.200740,0.034244,0.014176,0.066430
1277,(Schweppes),(Mahou),0.016640,0.056797,0.001094,0.065734,1.157359,1.0,0.000149,1.009566,0.138265,0.015120,0.009476,0.042496
95,(Casa Tarradellas),(Mahou),0.046161,0.056797,0.002839,0.061507,1.082939,1.0,0.000217,1.005019,0.080294,0.028359,0.004994,0.055749
275,(Fanta),(Mahou),0.026019,0.056797,0.001513,0.058140,1.023642,1.0,0.000035,1.001426,0.023713,0.018606,0.001424,0.042387
361,(Europastry),(Mahou),0.043229,0.056797,0.002281,0.052759,0.928910,1.0,-0.000175,0.995737,-0.074064,0.023333,-0.004281,0.046457
65,(Estrella Galicia),(Mahou),0.041437,0.056797,0.002129,0.051390,0.904806,1.0,-0.000224,0.994300,-0.098902,0.022158,-0.005732,0.044441
125,(Campofrio),(Mahou),0.031360,0.056797,0.001606,0.051206,0.901564,1.0,-0.000175,0.994107,-0.101300,0.018553,-0.005927,0.039739
718,(Lay's),(Mahou),0.046184,0.056797,0.002327,0.050391,0.887208,1.0,-0.000296,0.993254,-0.117611,0.023121,-0.006792,0.045683
141,(Coca-Cola),(Mahou),0.154798,0.056797,0.007156,0.046230,0.813958,1.0,-0.001636,0.988921,-0.212862,0.035005,-0.011203,0.086114


In [13]:
filtered_rules_marca = rules[rules['antecedents'].apply(lambda x: 'Heineken' in x)]

filtered_rules_marca = filtered_rules_marca.sort_values(by='confidence',ascending=False)

filtered_rules_marca.head(50)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
244,(Heineken),(Coca-Cola),0.013905,0.154798,0.001967,0.141423,0.913597,1.0,-0.000186,0.984422,-0.087515,0.011794,-0.015825,0.077063


In [18]:
# Filtrar reglas donde el antecedente sea exclusivamente "Mahou"
filtered_rules_marca_mahou = rules[rules['antecedents'].apply(lambda x: x == {'Solan'})]

# Ordenar por confianza en orden descendente
filtered_rules_marca_mahou = filtered_rules_marca_mahou.sort_values(by='confidence', ascending=False)

# Mostrar los primeros 50 resultados
filtered_rules_marca_mahou.head(50)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric


In [ ]:
filtered_rules_marca_mahou

In [14]:
# Filtrar reglas donde el antecedente sea exclusivamente "Mahou"
filtered_rules_marca_mahou = rules[rules['consequents'].apply(lambda x: x == {'Bud'})]

# Ordenar por confianza en orden descendente
filtered_rules_marca_mahou = filtered_rules_marca_mahou.sort_values(by='confidence', ascending=False)

# Mostrar los primeros 50 resultados
filtered_rules_marca_mahou.head(50)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
33967,(Coca-Cola),(Bud),0.156409,0.001468,0.000207,0.001325,0.902956,-0.000022,0.999857,-0.113004


In [22]:

query = """
select 
-- p_creation_date,
-- order_id,
-- customer_id,
--list_of_name,
list_of_product_brand,
-- count(order_id) as count_orders
order_id
-- *
from 

(SELECT
    bought_products.order_id  AS order_id,
    order_descriptors.customer_id  AS customer_id,
        (DATE_FORMAT(bought_products.p_creation_date , '%Y-%m-%d')) AS p_creation_date,
    ARRAY_JOIN(ARRAY_SORT(ARRAY_DISTINCT(ARRAY_AGG(bought_products.product_name ))),'*') AS "list_of_name",
    ARRAY_JOIN(ARRAY_SORT(ARRAY_DISTINCT(ARRAY_AGG(catalog_product_details.product_brand))),'*') AS "list_of_product_brand",
    ARRAY_JOIN(ARRAY_SORT(ARRAY_DISTINCT(ARRAY_AGG(catalog_product_details.product_category_level_two))),'*') AS "list_of_level_2",
    ARRAY_JOIN(ARRAY_SORT(ARRAY_DISTINCT(ARRAY_AGG(catalog_product_details.product_barcodes))),'|,|') AS list_sku
FROM delta.central__bought_products_looker__odp.bought_products  AS bought_products
LEFT JOIN delta.central_order_descriptors_odp.order_descriptors_v2  AS order_descriptors ON order_descriptors.order_id = bought_products.order_id
      AND order_descriptors.p_creation_date = bought_products.p_creation_date
LEFT JOIN delta.mfc_inventory_odp.products_v2  AS catalog_product_details ON CONCAT(CAST(bought_products.store_address_id AS VARCHAR), '_',
      bought_products.product_external_id) = (CONCAT(CAST(catalog_product_details.store_address_id AS VARCHAR), '_', catalog_product_details.product_sku))
WHERE
--  ((( bought_products.p_creation_date  ) >= ((DATE_ADD('day', -364, CAST(CAST(DATE_TRUNC('DAY', NOW()) AS DATE) AS TIMESTAMP)))) AND ( bought_products.p_creation_date  ) < ((DATE_ADD('day', 365, DATE_ADD('day', -364, CAST(CAST(DATE_TRUNC('DAY', NOW()) AS DATE) AS TIMESTAMP))))))) AND
 (order_descriptors.order_subvertical ) = 'MFC' AND (order_descriptors.order_country_code ) = 'ES' 
--  AND ((( order_descriptors.p_creation_date  ) >= ((DATE_ADD('month', -1, DATE_TRUNC('MONTH', CAST(CAST(DATE_TRUNC('DAY', NOW()) AS DATE) AS TIMESTAMP))))) AND ( order_descriptors.p_creation_date  ) < ((DATE_ADD('month', 1, DATE_ADD('month', -1, DATE_TRUNC('MONTH', CAST(CAST(DATE_TRUNC('DAY', NOW()) AS DATE) AS TIMESTAMP))))))))
AND ((( bought_products.p_creation_date  ) >= ((TIMESTAMP '2023-11-01')) AND ( bought_products.p_creation_date  ) < ((DATE_ADD('month', 12, TIMESTAMP '2023-11-01')))))
GROUP BY
    1,
    2,
    3
ORDER BY
    1 DESC)


where  
list_of_name like '%Budweiser%' 
--and list_of_product_brand like '%Diversual%'
and list_of_level_2 not like '%Fruit%' and list_of_level_2 not like '%Vegetables%' 
and list_of_product_brand not like  '%UNKNOWNBRAND%'
--and list_sku like '%8411327000079%' or list_sku like '%8411327000086%'
group by 1,2
"""

# Llama a la función `sql_dl` con la consulta SQL
df_marca = sql_dl(query)

# Muestra los datos obtenidos
df_marca.head(10)


,list_of_product_brand,order_id
0,Bud*Don*La Sirena*Micaderm,100756594284
1,Azpilicueta*Bud*Marqués de Riscal,100788831329
2,Bud,100654590595
3,Bud*Leffe*Mahou*TOP BEER*Xibeca,100487578750
4,Bud*Cheetos*Doritos*Dr Oetker*Europastry*Lay's...,100646907678
5,Bud*Corona,725619035
6,Aquarel*Bud,100392566522
7,Asturiana*Bud*Casa Forner*Navidul*Scottex,100647941840
8,Bud*Doritos,100821617906
9,Bud*Doritos,100585718509


In [23]:

from mlxtend.frequent_patterns import apriori, association_rules



# Separar las cadenas de texto en una lista de marcas
df_marca['list_of_product_brand'] = df_marca['list_of_product_brand'].apply(lambda x: x.split('*'))

# Crear una representación binaria (cada columna es una marca, cada fila un pedido)
from mlxtend.preprocessing import TransactionEncoder

# Usamos TransactionEncoder para convertir la lista de marcas en formato adecuado
te = TransactionEncoder()
te_ary = te.fit(df_marca['list_of_product_brand']).transform(df_marca['list_of_product_brand'])
df_binary = pd.DataFrame(te_ary, columns=te.columns_)



In [24]:
###FP GROWTH
from mlxtend.frequent_patterns import fpgrowth

# Aplicar FP-Growth con un soporte mínimo
frequent_itemsets = fpgrowth(df_binary, min_support=0.001, use_colnames=True)

# Mostrar los conjuntos frecuentes
frequent_itemsets.head(10)

,support,itemsets
0,1.000000,(Bud)
1,0.012085,(La Sirena)
2,0.010574,(Micaderm)
3,0.001511,(Don)
4,0.002266,(Marqués de Riscal)
5,0.001511,(Azpilicueta)
6,0.052115,(Mahou)
7,0.015106,(Xibeca)
8,0.004532,(TOP BEER)
9,0.003776,(Leffe)


In [ ]:
from mlxtend.frequent_patterns import association_rules

# Generar reglas de asociación basadas en los conjuntos frecuentes
rules = association_rules(frequent_itemsets, metric="lift")

# Ordenar las reglas por lift
rules = rules.sort_values(by='confidence', ascending=False)

# Mostrar las reglas más relevantes
rules.head(50)

In [20]:
# Filtrar reglas donde el antecedente sea exclusivamente "Mahou"
filtered_rules_marca_mahou = rules[rules['antecedents'].apply(lambda x: x == {'Pack 6x Budweiser Cerveza Lata 330ml'})]

# Ordenar por confianza en orden descendente
filtered_rules_marca_mahou = filtered_rules_marca_mahou.sort_values(by='lift', ascending=False)

# Mostrar los primeros 50 resultados
filtered_rules_marca_mahou.head(50)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
1561319,(Pack 6x Budweiser Cerveza Lata 330ml),"(Innocent Zumo Plus Rojo 330ml, Schweppes Refr...",0.444109,0.001511,0.001511,0.003401,2.251701,0.00084,1.001897,1.0
926808,(Pack 6x Budweiser Cerveza Lata 330ml),"(Innocent Zumo Plus Rojo 330ml, Haribo Berries...",0.444109,0.001511,0.001511,0.003401,2.251701,0.00084,1.001897,1.0
408060,(Pack 6x Budweiser Cerveza Lata 330ml),"(Pack 6x Damm Lemon Cerveza Lata 330ml, Schwep...",0.444109,0.001511,0.001511,0.003401,2.251701,0.00084,1.001897,1.0
4710,(Pack 6x Budweiser Cerveza Lata 330ml),"(Schweppes Refresco Limón 1L, Estrella Galicia...",0.444109,0.001511,0.001511,0.003401,2.251701,0.00084,1.001897,1.0
641285,(Pack 6x Budweiser Cerveza Lata 330ml),"(Schweppes Refresco Limón 1L, Haribo Berries M...",0.444109,0.001511,0.001511,0.003401,2.251701,0.00084,1.001897,1.0
881574,(Pack 6x Budweiser Cerveza Lata 330ml),"(Innocent Zumo Plus Rojo 330ml, Schweppes Refr...",0.444109,0.001511,0.001511,0.003401,2.251701,0.00084,1.001897,1.0
204318,(Pack 6x Budweiser Cerveza Lata 330ml),(Starbucks by Nespresso Café Descafeinado Espr...,0.444109,0.001511,0.001511,0.003401,2.251701,0.00084,1.001897,1.0
127100,(Pack 6x Budweiser Cerveza Lata 330ml),"(Pack 6x Xibeca Cerveza Lata 330ml, Pack 6x Da...",0.444109,0.001511,0.001511,0.003401,2.251701,0.00084,1.001897,1.0
854861,(Pack 6x Budweiser Cerveza Lata 330ml),"(Innocent Zumo Plus Rojo 330ml, Pack 6x Xibeca...",0.444109,0.001511,0.001511,0.003401,2.251701,0.00084,1.001897,1.0
195926,(Pack 6x Budweiser Cerveza Lata 330ml),"(Bimbo Pan de Molde Familiar con Corteza 770g,...",0.444109,0.001511,0.001511,0.003401,2.251701,0.00084,1.001897,1.0


In [21]:
# Filtrar reglas donde el antecedente sea exclusivamente "Mahou"
filtered_rules_marca_mahou = rules[rules['consequents'].apply(lambda x: x == {'Pack 6x Budweiser Cerveza Lata 330ml'})]

# Ordenar por confianza en orden descendente
filtered_rules_marca_mahou = filtered_rules_marca_mahou.sort_values(by='support', ascending=False)

# Mostrar los primeros 50 resultados
filtered_rules_marca_mahou.head(50)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
2786,(Hielo Cubitos Normales 2kg),(Pack 6x Budweiser Cerveza Lata 330ml),0.053625,0.444109,0.024169,0.450704,1.014851,0.000354,1.012007,0.015463
1741,"(Pack Bezoya 1,5L Agua Mineral)",(Pack 6x Budweiser Cerveza Lata 330ml),0.026435,0.444109,0.012840,0.485714,1.093683,0.001100,1.080900,0.087984
187500,(Aquarel 5L Agua Mineral),(Pack 6x Budweiser Cerveza Lata 330ml),0.015861,0.444109,0.011329,0.714286,1.608358,0.004285,1.945619,0.384344
151180,(Pack 6x Corona Cerveza Botella 350ml),(Pack 6x Budweiser Cerveza Lata 330ml),0.008308,0.444109,0.008308,1.000000,2.251701,0.004618,inf,0.560548
142484,(Pack 6x Estrella Galicia Cerveza Lata 330ml),(Pack 6x Budweiser Cerveza Lata 330ml),0.010574,0.444109,0.008308,0.785714,1.769193,0.003612,2.594159,0.439417
1429,(Estrella Galicia Cerveza Lata 330ml),(Pack 6x Budweiser Cerveza Lata 330ml),0.013595,0.444109,0.007553,0.555556,1.250945,0.001515,1.250755,0.203369
142912,(Pack 8x Heineken Cerveza Lata 330ml),(Pack 6x Budweiser Cerveza Lata 330ml),0.008308,0.444109,0.007553,0.909091,2.047001,0.003863,6.114804,0.515765
1836,(Asturiana Leche sin Lactosa Semidesnatada 1L),(Pack 6x Budweiser Cerveza Lata 330ml),0.012840,0.444109,0.007553,0.588235,1.324530,0.001851,1.350022,0.248202
142922,(Pack 6x Damm Lemon Cerveza Lata 330ml),(Pack 6x Budweiser Cerveza Lata 330ml),0.007553,0.444109,0.006798,0.900000,2.026531,0.003443,5.558912,0.510401
141077,"(Aquarius Limón Botella 1,5L)",(Pack 6x Budweiser Cerveza Lata 330ml),0.009819,0.444109,0.006042,0.615385,1.385662,0.001682,1.445317,0.281083
